# MTA Analysis: Data Acquisition & Cleaning

This analysis uses [publically available MTA Turnstile Data](http://web.mta.info/developers/turnstile.html) ranging from January 2018 until June 2020.

#### Import required libraries.

In [ ]:
import pandas as pd
import urllib

#### Define useful functions (maybe document and put this in .py file later?)

In [ ]:
def convert_timestamp_to_mta_format(timestamp):
    
    year = str(timestamp.year)[2:4]
    month = str(timestamp.month).zfill(2)
    day = str(timestamp.day).zfill(2)
    
    date_mta_format = year + month + day
    
    return date_mta_format

#### Load [MTA Turnstile Data](http://web.mta.info/developers/turnstile.html) into CSV files. 
_Note: The files already exist in the repository. This code block can be skipped over._

In [ ]:
start_date = '2018-01-06'
start_ts = pd.Timestamp(start_date)

In [ ]:
current_timestamp = start_ts
is_date_valid = True
while is_date_valid:
    date_formatted = convert_timestamp_to_mta_format(current_timestamp)
    
    # load data and write to csv
    url = f'http://web.mta.info/developers/data/nyct/turnstile/turnstile_{date_formatted}.txt'
    df_turnstile_data = pd.read_csv(url)
    df_turnstile_data.to_csv(f'raw_data/mta_turnstile_data/raw_mta_turnstile_data_{date_formatted}.csv')
    
    # add 7 days to get next file. if resulting date is later than today, then stop loop
    current_timestamp += pd.DateOffset(days=7)
    if current_timestamp > pd.to_datetime("now"):
        is_data_valid = False